In [24]:
import pickle
import pathlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [25]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

/home/fernando/6/machinelearning/machine_learning/CSRoundPrediction/data


In [26]:
clean_data_path = DATA_DIR / 'processed' / 'csgo_clean.pkl'

In [27]:
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122410 entries, 0 to 122409
Data columns (total 70 columns):
 #   Column                        Non-Null Count   Dtype   
---  ------                        --------------   -----   
 0   time_left                     122410 non-null  float64 
 1   ct_score                      122410 non-null  float64 
 2   t_score                       122410 non-null  float64 
 3   map                           122410 non-null  category
 4   bomb_planted                  122410 non-null  bool    
 5   ct_health                     122410 non-null  float64 
 6   t_health                      122410 non-null  float64 
 7   ct_armor                      122410 non-null  float64 
 8   t_armor                       122410 non-null  float64 
 9   ct_money                      122410 non-null  float64 
 10  t_money                       122410 non-null  float64 
 11  ct_helmets                    122410 non-null  float64 
 12  t_helmets                     

In [29]:
model_data = data.copy()

In [30]:
categorical_columns = ['map']

## Encoding Categorical Variable

In [31]:
model_data['map'].value_counts()

map
de_inferno     23811
de_dust2       22144
de_nuke        19025
de_mirage      18576
de_overpass    14081
de_train       13491
de_vertigo     11137
de_cache         145
Name: count, dtype: int64

In [32]:
original_data = model_data['map']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['map'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
de_dust2,True,True,True,True,True
de_inferno,False,False,False,False,False
de_mirage,False,False,False,False,False
de_nuke,False,False,False,False,False
de_overpass,False,False,False,False,False
de_train,False,False,False,False,False
de_vertigo,False,False,False,False,False
map,de_dust2,de_dust2,de_dust2,de_dust2,de_dust2


In [33]:
model_data = pd.get_dummies(model_data, drop_first=True)

In [34]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122410 entries, 0 to 122409
Data columns (total 76 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   time_left                     122410 non-null  float64
 1   ct_score                      122410 non-null  float64
 2   t_score                       122410 non-null  float64
 3   bomb_planted                  122410 non-null  bool   
 4   ct_health                     122410 non-null  float64
 5   t_health                      122410 non-null  float64
 6   ct_armor                      122410 non-null  float64
 7   t_armor                       122410 non-null  float64
 8   ct_money                      122410 non-null  float64
 9   t_money                       122410 non-null  float64
 10  ct_helmets                    122410 non-null  float64
 11  t_helmets                     122410 non-null  float64
 12  ct_defuse_kits                122410 non-nul

## Train test split

In [35]:
X = model_data.drop(columns=['round_winner']).copy()
y = model_data['round_winner'].copy()

In [36]:
X.values, y.values

(array([[175.0, 0.0, 0.0, ..., False, False, False],
        [156.03, 0.0, 0.0, ..., False, False, False],
        [96.03, 0.0, 0.0, ..., False, False, False],
        ...,
        [114.93, 11.0, 15.0, ..., False, True, False],
        [94.93, 11.0, 15.0, ..., False, True, False],
        [74.93, 11.0, 15.0, ..., False, True, False]], dtype=object),
 array([1, 1, 1, ..., 0, 0, 0]))

In [37]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=69,
)


In [38]:
X.shape, Xtrain.shape, Xtest.shape

((122410, 75), (91807, 75), (30603, 75))

In [39]:
y.shape, ytrain.shape, ytest.shape

((122410,), (91807,), (30603,))

In [40]:
# 0.74 Mas não converge
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(Xtrain, ytrain)

/home/fernando/6/machinelearning/machine_learning/CSRoundPrediction/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# 0.74
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=69,
)

model.fit(Xtrain, ytrain)

In [ ]:
# 0.76
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()

model.fit(Xtrain, ytrain)

In [ ]:
# O melhor que eu consegui mexendo nos parametros foi 0.76
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(
    hidden_layer_sizes=(64, 32),  # Two hidden layers with 64 and 32 neurons
    activation='relu',            # Use ReLU activation function
    solver='adam',                # Use Adam optimizer
    learning_rate='constant',     # Adaptive learning rate
    random_state=42               # Ensure reproducibility
)

model.fit(Xtrain, ytrain)

In [41]:
# 0.82
from sklearn import tree

model = tree.DecisionTreeClassifier()

model.fit(Xtrain, ytrain)

DecisionTreeClassifier()

In [ ]:
# 0.72
from sklearn import svm

model = svm.SVC() # AINDA NAO

model.fit(Xtrain, ytrain)

In [42]:
ypred = model.predict(Xtest)
ypred

array([1, 1, 0, ..., 0, 0, 1])

In [43]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(ytest, ypred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.82


This is a incredibly good result, now let's look at other cenarios.

## Other cenarios

### 1. Splitting the data between the start, middle and end of the round (looking at the time left)

In [48]:
data_start = model_data[model_data['time_left'] > 125]
data_middle = model_data[(model_data['time_left'] <= 125) & (model_data['time_left'] > 75)]
data_end = model_data[model_data['time_left'] <= 75]

In [49]:
x_start = data_start.drop(columns=['round_winner']).copy()
y_start = data_start['round_winner'].copy()

x_middle = data_middle.drop(columns=['round_winner']).copy()
y_middle = data_middle['round_winner'].copy()

x_end = data_end.drop(columns=['round_winner']).copy()
y_end = data_end['round_winner'].copy()

In [50]:
x_start_train, x_start_test, y_start_train, y_start_test = train_test_split(
    x_start,
    y_start,
    test_size=0.25,
    random_state=69,
)

x_middle_train, x_middle_test, y_middle_train, y_middle_test = train_test_split(
    x_middle,
    y_middle,
    test_size=0.25,
    random_state=69,
)

x_end_train, x_end_test, y_end_train, y_end_test = train_test_split(
    x_end,
    y_end,
    test_size=0.25,
    random_state=69,
)

In [51]:
model_start = tree.DecisionTreeClassifier()
model_middle = tree.DecisionTreeClassifier()
model_end = tree.DecisionTreeClassifier()

model_start.fit(x_start_train, y_start_train)
model_middle.fit(x_middle_train, y_middle_train)
model_end.fit(x_end_train, y_end_train)

DecisionTreeClassifier()

In [52]:
ypred_start = model_start.predict(x_start_test)
ypred_middle = model_middle.predict(x_middle_test)
ypred_end = model_end.predict(x_end_test)

In [53]:
accuracy_start = accuracy_score(y_start_test, ypred_start)
accuracy_middle = accuracy_score(y_middle_test, ypred_middle)
accuracy_end = accuracy_score(y_end_test, ypred_end)

print(f"Start Accuracy: {accuracy_start:.2f}")
print(f"Middle Accuracy: {accuracy_middle:.2f}")
print(f"End Accuracy: {accuracy_end:.2f}")

Start Accuracy: 0.88
Middle Accuracy: 0.69
End Accuracy: 0.81


Here we can see another interesting result, the model is able to predict the outcome of the round with a much higher accuracy when the round is beggining, this is probably due to the fact that rounds that are meant to be one sided and easy to predict, do tend to end more quickly, while rounds that could have a random factor and be very hard, drag on for longer.

### 2. Splitting the data for clutch situations

This is an interesting one, a clutch is a situation where only one player on one of the teams is alive, let's take a look at these situations. This should be much easier for the model to predict on situations of a difficult clutch, like a 1v5, since the outcome is much more predictable, however, on a 1v1, the outcome is much more random, so the model should have a harder time predicting the outcome.

In [57]:
data_ct_1v5 = model_data[(model_data['ct_players_alive'] == 1) & (model_data['t_players_alive'] == 5)]
data_ct_1v4 = model_data[(model_data['ct_players_alive'] == 1) & (model_data['t_players_alive'] == 4)]
data_ct_1v3 = model_data[(model_data['ct_players_alive'] == 1) & (model_data['t_players_alive'] == 3)]
data_ct_1v2 = model_data[(model_data['ct_players_alive'] == 1) & (model_data['t_players_alive'] == 2)]
data_ct_1v1 = model_data[(model_data['ct_players_alive'] == 1) & (model_data['t_players_alive'] == 1)]
data_t_1v5 = model_data[(model_data['ct_players_alive'] == 5) & (model_data['t_players_alive'] == 1)]
data_t_1v4 = model_data[(model_data['ct_players_alive'] == 4) & (model_data['t_players_alive'] == 1)]
data_t_1v3 = model_data[(model_data['ct_players_alive'] == 3) & (model_data['t_players_alive'] == 1)]
data_t_1v2 = model_data[(model_data['ct_players_alive'] == 2) & (model_data['t_players_alive'] == 1)]
data_t_1v1 = model_data[(model_data['ct_players_alive'] == 1) & (model_data['t_players_alive'] == 1)]

In [58]:
x_ct_1v5 = data_ct_1v5.drop(columns=['round_winner']).copy()
y_ct_1v5 = data_ct_1v5['round_winner'].copy()

x_ct_1v4 = data_ct_1v4.drop(columns=['round_winner']).copy()
y_ct_1v4 = data_ct_1v4['round_winner'].copy()

x_ct_1v3 = data_ct_1v3.drop(columns=['round_winner']).copy()
y_ct_1v3 = data_ct_1v3['round_winner'].copy()

x_ct_1v2 = data_ct_1v2.drop(columns=['round_winner']).copy()
y_ct_1v2 = data_ct_1v2['round_winner'].copy()

x_ct_1v1 = data_ct_1v1.drop(columns=['round_winner']).copy()
y_ct_1v1 = data_ct_1v1['round_winner'].copy()

x_t_1v5 = data_t_1v5.drop(columns=['round_winner']).copy()
y_t_1v5 = data_t_1v5['round_winner'].copy()

x_t_1v4 = data_t_1v4.drop(columns=['round_winner']).copy()
y_t_1v4 = data_t_1v4['round_winner'].copy()

x_t_1v3 = data_t_1v3.drop(columns=['round_winner']).copy()
y_t_1v3 = data_t_1v3['round_winner'].copy()

x_t_1v2 = data_t_1v2.drop(columns=['round_winner']).copy()
y_t_1v2 = data_t_1v2['round_winner'].copy()

x_t_1v1 = data_t_1v1.drop(columns=['round_winner']).copy()
y_t_1v1 = data_t_1v1['round_winner'].copy()

In [59]:
x_ct_1v5_train, x_ct_1v5_test, y_ct_1v5_train, y_ct_1v5_test = train_test_split(
    x_ct_1v5,
    y_ct_1v5,
    test_size=0.25,
    random_state=69,
)

x_ct_1v4_train, x_ct_1v4_test, y_ct_1v4_train, y_ct_1v4_test = train_test_split(
    x_ct_1v4,
    y_ct_1v4,
    test_size=0.25,
    random_state=69,
)

x_ct_1v3_train, x_ct_1v3_test, y_ct_1v3_train, y_ct_1v3_test = train_test_split(
    x_ct_1v3,
    y_ct_1v3,
    test_size=0.25,
    random_state=69,
)

x_ct_1v2_train, x_ct_1v2_test, y_ct_1v2_train, y_ct_1v2_test = train_test_split(
    x_ct_1v2,
    y_ct_1v2,
    test_size=0.25,
    random_state=69,
)

x_ct_1v1_train, x_ct_1v1_test, y_ct_1v1_train, y_ct_1v1_test = train_test_split(
    x_ct_1v1,
    y_ct_1v1,
    test_size=0.25,
    random_state=69,
)

x_t_1v5_train, x_t_1v5_test, y_t_1v5_train, y_t_1v5_test = train_test_split(
    x_t_1v5,
    y_t_1v5,
    test_size=0.25,
    random_state=69,
)

x_t_1v4_train, x_t_1v4_test, y_t_1v4_train, y_t_1v4_test = train_test_split(
    x_t_1v4,
    y_t_1v4,
    test_size=0.25,
    random_state=69,
)

x_t_1v3_train, x_t_1v3_test, y_t_1v3_train, y_t_1v3_test = train_test_split(
    x_t_1v3,
    y_t_1v3,
    test_size=0.25,
    random_state=69,
)

x_t_1v2_train, x_t_1v2_test, y_t_1v2_train, y_t_1v2_test = train_test_split(
    x_t_1v2,
    y_t_1v2,
    test_size=0.25,
    random_state=69,
)

x_t_1v1_train, x_t_1v1_test, y_t_1v1_train, y_t_1v1_test = train_test_split(
    x_t_1v1,
    y_t_1v1,
    test_size=0.25,
    random_state=69,
)

In [60]:
model_ct_1v5 = tree.DecisionTreeClassifier()
model_ct_1v4 = tree.DecisionTreeClassifier()
model_ct_1v3 = tree.DecisionTreeClassifier()    
model_ct_1v2 = tree.DecisionTreeClassifier()
model_ct_1v1 = tree.DecisionTreeClassifier()
model_t_1v5 = tree.DecisionTreeClassifier()
model_t_1v4 = tree.DecisionTreeClassifier()
model_t_1v3 = tree.DecisionTreeClassifier()
model_t_1v2 = tree.DecisionTreeClassifier()
model_t_1v1 = tree.DecisionTreeClassifier()

model_ct_1v5.fit(x_ct_1v5_train, y_ct_1v5_train)
model_ct_1v4.fit(x_ct_1v4_train, y_ct_1v4_train)
model_ct_1v3.fit(x_ct_1v3_train, y_ct_1v3_train)
model_ct_1v2.fit(x_ct_1v2_train, y_ct_1v2_train)
model_ct_1v1.fit(x_ct_1v1_train, y_ct_1v1_train)
model_t_1v5.fit(x_t_1v5_train, y_t_1v5_train)
model_t_1v4.fit(x_t_1v4_train, y_t_1v4_train)
model_t_1v3.fit(x_t_1v3_train, y_t_1v3_train)
model_t_1v2.fit(x_t_1v2_train, y_t_1v2_train)
model_t_1v1.fit(x_t_1v1_train, y_t_1v1_train)

DecisionTreeClassifier()

In [61]:
ypred_ct_1v5 = model_ct_1v5.predict(x_ct_1v5_test)
ypred_ct_1v4 = model_ct_1v4.predict(x_ct_1v4_test)
ypred_ct_1v3 = model_ct_1v3.predict(x_ct_1v3_test)
ypred_ct_1v2 = model_ct_1v2.predict(x_ct_1v2_test)
ypred_ct_1v1 = model_ct_1v1.predict(x_ct_1v1_test)
ypred_t_1v5 = model_t_1v5.predict(x_t_1v5_test)
ypred_t_1v4 = model_t_1v4.predict(x_t_1v4_test)
ypred_t_1v3 = model_t_1v3.predict(x_t_1v3_test)
ypred_t_1v2 = model_t_1v2.predict(x_t_1v2_test)
ypred_t_1v1 = model_t_1v1.predict(x_t_1v1_test)

In [64]:
accuracy_ct_1v5 = accuracy_score(y_ct_1v5_test, ypred_ct_1v5)
accuracy_ct_1v4 = accuracy_score(y_ct_1v4_test, ypred_ct_1v4)
accuracy_ct_1v3 = accuracy_score(y_ct_1v3_test, ypred_ct_1v3)
accuracy_ct_1v2 = accuracy_score(y_ct_1v2_test, ypred_ct_1v2)
accuracy_ct_1v1 = accuracy_score(y_ct_1v1_test, ypred_ct_1v1)
accuracy_t_1v5 = accuracy_score(y_t_1v5_test, ypred_t_1v5)
accuracy_t_1v4 = accuracy_score(y_t_1v4_test, ypred_t_1v4)
accuracy_t_1v3 = accuracy_score(y_t_1v3_test, ypred_t_1v3)
accuracy_t_1v2 = accuracy_score(y_t_1v2_test, ypred_t_1v2)
accuracy_t_1v1 = accuracy_score(y_t_1v1_test, ypred_t_1v1)

print(f"CT 1v5 Accuracy: {accuracy_ct_1v5:.2f}")
print(f"CT 1v4 Accuracy: {accuracy_ct_1v4:.2f}")
print(f"CT 1v3 Accuracy: {accuracy_ct_1v3:.2f}")
print(f"CT 1v2 Accuracy: {accuracy_ct_1v2:.2f}")
print(f"CT 1v1 Accuracy: {accuracy_ct_1v1:.2f}")
print("---------------------")
print(f"T 1v5 Accuracy: {accuracy_t_1v5:.2f}")
print(f"T 1v4 Accuracy: {accuracy_t_1v4:.2f}")
print(f"T 1v3 Accuracy: {accuracy_t_1v3:.2f}")
print(f"T 1v2 Accuracy: {accuracy_t_1v2:.2f}")
print(f"T 1v1 Accuracy: {accuracy_t_1v1:.2f}")

CT 1v5 Accuracy: 1.00
CT 1v4 Accuracy: 1.00
CT 1v3 Accuracy: 0.98
CT 1v2 Accuracy: 0.84
CT 1v1 Accuracy: 0.65
---------------------
T 1v5 Accuracy: 1.00
T 1v4 Accuracy: 0.99
T 1v3 Accuracy: 0.95
T 1v2 Accuracy: 0.77
T 1v1 Accuracy: 0.69


### 3. Economical rounds

### 4. Pistol rounds

### 5. Rounds with a bomb plant